## Importing Libraries and Dataframe loading

In [72]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, train_test_split, RandomizedSearchCV
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from itertools import combinations 
from scipy.stats import ttest_rel
from sklearn import tree, svm
import pandas as pd
import numpy as np
import math

# The file does not use commas as a separator, the "sep" argument helps by setting the tab character as the separator
# The file does not have a header row, the "names" argument is used to define the header for each column
df = pd.read_csv("smsspamcollection/SMSSpamCollection.csv", sep="\t", names=["Type", "Text"], header=None)

## DataFrame Analysis

In [54]:
# Print the first elements of the DataFrame
print(df.head())
# Provide information about the DataFrame columns
print(df.info())
# Give a brief description of the DataFrame data
print(df.describe())


   Type                                               Text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Type    5572 non-null   object
 1   Text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB
None
        Type                    Text
count   5572                    5572
unique     2                    5169
top      ham  Sorry, I'll call later
freq    4825                      30


In [55]:
# The "Type" column has values "ham" and "spam", so I changed it to "categorical"
# More information about pandas "categorical" can be found at: https://pandas.pydata.org/docs/reference/api/pandas.Categorical.html
df['Type'] = df['Type'].astype('category')


## TF-IDF (Term Frequency-Inverse Document Frequency)

The formula for TF-IDF is:  

(term frequency in the document) × log((total number of documents) / (number of documents containing the term)).

I implemented the algorithm to test:


In [56]:
# This can be simplified, but I did it just for testing (a class would also be good)
tf_count = {}
num_documents = 0

for index, row in df.iterrows():
    line = row['Text'].strip().split()
    num_documents += 1
    seen = set()
    for word in line:
        if word not in seen:
            if word in tf_count:
                tf_count[word] += 1
            else:
                tf_count[word] = 1
        seen.add(word)

res = []
for index, row in df.iterrows():
    line = row['Text'].strip().split()
    local_res = []
    local_m = {}
    for word in line:
        if word in local_m:
            local_m[word] += 1
        else:
            local_m[word] = 1
    for word in line:
        local_res.append((word, local_m[word] * math.log(num_documents/1e-3 + tf_count[word])))
    res.append(local_res)

print(res[0])

[('Go', 15.53326712644149), ('until', 15.53326928005903), ('jurong', 15.53326479335059), ('point,', 15.53326479335059), ('crazy..', 15.53326479335059), ('Available', 15.533265152288006), ('only', 15.533290098122793), ('in', 15.533394720277643), ('bugis', 15.533265331756665), ('n', 15.533285252577684), ('great', 15.533277176616993), ('world', 15.53326766484631), ('la', 15.533264972819314), ('e', 15.533276279284001), ('buffet...', 15.53326479335059), ('Cine', 15.53326479335059), ('there', 15.533283816856105), ('got', 15.533299789142573), ('amore', 15.53326479335059), ('wat...', 15.533266588036382)]


I’m going to end up using Scikit-learn’s TF-IDF implementation:  

reference: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [57]:
vectorizer   = TfidfVectorizer()
tfidx_matrix = vectorizer.fit_transform(df['Text'])
tfidx_df     = pd.DataFrame(tfidx_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidx_df.describe()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
count,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,...,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000
mean,0.000402,0.001161,0.000042,0.000094,0.000045,0.000055,0.000052,0.000082,0.000092,0.000352,...,0.000049,0.000058,0.000104,0.000064,0.000052,0.000049,0.000029,0.000033,0.000040,0.000055
std,0.009510,0.018111,0.003123,0.004943,0.003352,0.004083,0.003888,0.004314,0.006843,0.009281,...,0.003669,0.004299,0.005467,0.004746,0.003886,0.003670,0.002150,0.002476,0.003011,0.004138
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.240128,0.654425,0.233155,0.265767,0.250210,0.304792,0.290193,0.227745,0.510769,0.256937,...,0.273889,0.320935,0.296818,0.354245,0.290067,0.273938,0.160467,0.184833,0.224784,0.308912


## Dataset train/test split

This section is easy using Scikit implementations, but later will get a bit harder with K-fold

In [58]:
X = tfidx_df
Y = df['Type']
x_train, x_test, y_train, y_test = train_test_split(X, Y)

## Classification (3 models)

### Decision Tree:

Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. A tree can be seen as a piecewise constant approximation.

Statquest's video:
https://www.youtube.com/watch?v=_L39rN6gz7Y&t=429s

Scikit-Learn:  
https://scikit-learn.org/stable/modules/tree.html


In [ ]:
model_dtree = tree.DecisionTreeClassifier()
model_dtree = model_dtree.fit(x_train, y_train)

y_pred = model_dtree.predict(x_test)
print(accuracy_score(y_pred, y_test))
print(f1_score(y_pred, y_test, average='macro'))


"\nparameters = {\n    'max_depth': [5, 10, 20, 50],\n    'min_samples_split': [2, 5, 10],\n    'min_samples_leaf': [1, 2, 4],\n}\n\nmodel_dtree = GridSearchCV(model_dtree, parameters)\nmodel_dtree = model_dtree.fit(x_train, y_train)\n\ny_pred = model_dtree.predict(x_test)\naccuracy_score(y_pred, y_test)\nf1_score(y_pred, y_test, average='macro')\n"

### Support Vector Machines:

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

It tries to find the best boundary known as hyperplane that separates different classes in the data.

Statequest's videos:  
https://www.youtube.com/watch?v=efR1C6CvhmE  
https://www.youtube.com/watch?v=Toet3EiSFcM  
https://www.youtube.com/watch?v=Qc5IyLW_hns

In [60]:
model_svm = svm.SVC()
model_svm.fit(x_train, y_train)

y_pred = model_svm.predict(x_test)
print(accuracy_score(y_pred, y_test))
print(f1_score(y_pred, y_test, average='macro'))

0.9820531227566404
0.9609529607915028


### K Nearest Neighbours

The principle behind nearest neighbor methods is to find a predefined number of training samples closest in distance to the new point, and predict the label from these. The number of samples can be a user-defined constant (k-nearest neighbor learning), or vary based on the local density of points (radius-based neighbor learning). The distance can, in general, be any metric measure: standard Euclidean distance is the most common choice. Neighbors-based methods are known as non-generalizing machine learning methods, since they simply “remember” all of its training data (possibly transformed into a fast indexing structure such as a Ball Tree or KD Tree).


Statquest's video:
https://www.youtube.com/watch?v=HVXime0nQeI

In [61]:
model_knn = KNeighborsClassifier(n_neighbors = 4)
model_knn.fit(x_train, y_train)

y_pred = model_knn.predict(x_test)
accuracy_score(y_pred, y_test)
f1_score(y_pred, y_test, average='macro')

0.6967633192770335

#### K-Fold Cross-Validation

1. Shuffle the dataset randomly.  
2. Split the dataset into *k* groups.  
3. For each unique group:  
   1. Use the group as a test set (*hold out*).  
   2. Use the remaining groups as the training set.  
   3. Train a model on the training set and evaluate it on the test set.  
   4. Save the evaluation score and discard the model.  
4. Summarize the model’s performance using the sample of evaluation scores obtained.

### K-Fold

Provides training/testing indices to split the data into training and test sets.  
Splits the dataset into *k* consecutive folds (without shuffling by default).  
Each fold is used once as validation, while the remaining *k - 1* folds form the training set.

In [62]:
skf = KFold(n_splits=4)
skf.get_n_splits(X, Y)

for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):
    print(f'{i}: train={train_idx}, test={test_idx}')

0: train=[1393 1394 1395 ... 5569 5570 5571], test=[   0    1    2 ... 1390 1391 1392]
1: train=[   0    1    2 ... 5569 5570 5571], test=[1393 1394 1395 ... 2783 2784 2785]
2: train=[   0    1    2 ... 5569 5570 5571], test=[2786 2787 2788 ... 4176 4177 4178]
3: train=[   0    1    2 ... 4176 4177 4178], test=[4179 4180 4181 ... 5569 5570 5571]


### Stratified K-Fold

This cross-validation object is a variation of K-Fold that returns stratified folds.  
The folds are created by preserving the percentage of samples for each class in *y* in a binary or multiclass classification scenario.

In [63]:
skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(X, Y)

for i, (train_idx, test_idx) in enumerate(skf.split(X, Y)):
    print(f'{i}: train={train_idx}, test={test_idx}')

0: train=[1227 1229 1252 ... 5569 5570 5571], test=[   0    1    2 ... 1406 1408 1409]
1: train=[   0    1    2 ... 5569 5570 5571], test=[1227 1229 1252 ... 2792 2793 2794]
2: train=[   0    1    2 ... 5569 5570 5571], test=[2719 2729 2730 ... 4181 4182 4184]
3: train=[   0    1    2 ... 4181 4182 4184], test=[4154 4156 4162 ... 5569 5570 5571]


### Grid-Search

Exhaustive search over specified parameter values for an estimator.

In [ ]:
# This is too slow to let it run
'''
parameters = {
    'max_depth': [5, 10, 20, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

model_dtree = GridSearchCV(model_dtree, parameters)
model_dtree = model_dtree.fit(x_train, y_train)

y_pred = model_dtree.predict(x_test)
accuracy_score(y_pred, y_test)
f1_score(y_pred, y_test, average='macro')
'''


### Randomized Search cv

Randomized search on hyper parameters.

In [ ]:
'''
parameters = {
    'max_depth':         randint(5, 50),           
    'min_samples_split': randint(2, 20),   
    'min_samples_leaf':  randint(1, 10),    
}

model_dtree = RandomizedSearchCV(model_dtree, parameters)
search = model_dtree.fit(x_train, y_train)
search.best_params_
'''

#### Comparing randomized search and grid search for hyperparameter estimation
https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html

### T-Test

The t-test is a statistical tool used to determine whether there is a significant difference between the means of two groups, or between the mean of a group and a known value.

Video: https://www.youtube.com/watch?v=VekJxtk4BYM

#### One Sample

t = (x̄ - μ) / (s / sqrt(n))

x̄ = sample mean  
μ = assumed (or population) mean  
s = sample standard deviation  
n = number of observations (sample size)

#### Two Samples

t = (x̄₁ - x̄₂) / sqrt((s₁² / n₁) + (s₂² / n₂))

x̄₁ = observed mean of the 1st sample  
x̄₂ = observed mean of the 2nd sample  
s₁ = standard deviation of the 1st sample  
s₂ = standard deviation of the 2nd sample  
n₁ = size of the 1st sample  
n₂ = size of the 2nd sample  

#### Bonferroni Correction

A statistical method used to control the false positive rate (Type I errors) when performing multiple statistical tests on the same dataset.  
If you perform *m* independent tests with a total significance level *α*, the Bonferroni-corrected threshold is:  

alpha_corrected = alpha / m  

A test is considered significant if:  

p_i < alpha_corrected  

Alternatively, you can adjust the p-values directly:  

p_adjusted = p_i * m  

Then, compare the adjusted p-values with the original significance level (alpha).


### Final test

In [64]:
kfold = StratifiedKFold(n_splits=10)
kfold.get_n_splits(X, Y)

model_dtree = tree.DecisionTreeClassifier()
model_svm = svm.SVC()
model_knear = KNeighborsClassifier()

acc_dtree, acc_svm, acc_knear = [], [], []

'''
for i, (train_idx, test_idx) in enumerate(kfold.split(X, Y)):
    # Variables for the "blocks"
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]

    # Train the models
    model_dtree.fit(X_train, y_train)
    model_svm.fit(X_train, y_train)
    model_knear.fit(X_train, y_train)

    # Make predictions for y
    y_pred_dtree = model_dtree.predict(X_test)
    y_pred_svm   = model_svm.predict(X_test)
    y_pred_knear = model_knear.predict(X_test)

    # Save the accuracy
    acc_dtree.append(accuracy_score(y_test, y_pred_dtree))
    acc_svm.append(accuracy_score(y_test, y_pred_svm))
    acc_knear.append(accuracy_score(y_test, y_pred_knear))
'''

'\nfor i, (train_idx, test_idx) in enumerate(kfold.split(X, Y)):\n    # Variables for the "blocks"\n    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]\n    y_train, y_test = Y.iloc[train_idx], Y.iloc[test_idx]\n\n    # Train the models\n    model_dtree.fit(X_train, y_train)\n    model_svm.fit(X_train, y_train)\n    model_knear.fit(X_train, y_train)\n\n    # Make predictions for y\n    y_pred_dtree = model_dtree.predict(X_test)\n    y_pred_svm   = model_svm.predict(X_test)\n    y_pred_knear = model_knear.predict(X_test)\n\n    # Save the accuracy\n    acc_dtree.append(accuracy_score(y_test, y_pred_dtree))\n    acc_svm.append(accuracy_score(y_test, y_pred_svm))\n    acc_knear.append(accuracy_score(y_test, y_pred_knear))\n'

Using Scipy T-test without correction:

In [65]:
# Compare models using T-test
t_dtree_svm, p_dtree_svm     = ttest_rel(acc_dtree, acc_svm)
t_dtree_knear, p_dtree_knear = ttest_rel(acc_dtree, acc_knear)
t_svm_knear, p_svm_knear     = ttest_rel(acc_svm, acc_knear)

print("Decision Tree vs SVM: t = %.3f, p = %.3f" % (t_dtree_svm, p_dtree_svm))
print("Decision Tree vs KNN: t = %.3f, p = %.3f" % (t_dtree_knear, p_dtree_knear))
print("SVM vs KNN:           t = %.3f, p = %.3f" % (t_svm_knear, p_svm_knear))


Decision Tree vs SVM: t = nan, p = nan
Decision Tree vs KNN: t = nan, p = nan
SVM vs KNN:           t = nan, p = nan


/var/folders/xb/dnjxhntx75jbglwhpt5wvl4h0000gn/T/ipykernel_39770/3569224788.py:2: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_dtree_svm, p_dtree_svm     = ttest_rel(acc_dtree, acc_svm)
/var/folders/xb/dnjxhntx75jbglwhpt5wvl4h0000gn/T/ipykernel_39770/3569224788.py:3: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_dtree_knear, p_dtree_knear = ttest_rel(acc_dtree, acc_knear)
/var/folders/xb/dnjxhntx75jbglwhpt5wvl4h0000gn/T/ipykernel_39770/3569224788.py:4: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_svm_knear, p_svm_knear     = ttest_rel(acc_svm, acc_knear)


with correction:

In [66]:
alpha = 0.05
# We have 3 comparisons
alpha_bonf = alpha / 3

t_dtree_svm, p_dtree_svm     = ttest_rel(acc_dtree, acc_svm)
t_dtree_knear, p_dtree_knear = ttest_rel(acc_dtree, acc_knear)
t_svm_knear, p_svm_knear     = ttest_rel(acc_svm, acc_knear)

print("Decision Tree vs SVM: t = %.3f, p = %.3f, significant = %s" %
      (t_dtree_svm, p_dtree_svm, p_dtree_svm < alpha_bonf))
print("Decision Tree vs KNN: t = %.3f, p = %.3f, significant = %s" %
      (t_dtree_knear, p_dtree_knear, p_dtree_knear < alpha_bonf))
print("SVM vs KNN: t = %.3f, p = %.3f, significant = %s" %
      (t_svm_knear, p_svm_knear, p_svm_knear < alpha_bonf))


Decision Tree vs SVM: t = nan, p = nan, significant = False
Decision Tree vs KNN: t = nan, p = nan, significant = False
SVM vs KNN: t = nan, p = nan, significant = False


/var/folders/xb/dnjxhntx75jbglwhpt5wvl4h0000gn/T/ipykernel_39770/3968015007.py:5: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_dtree_svm, p_dtree_svm     = ttest_rel(acc_dtree, acc_svm)
/var/folders/xb/dnjxhntx75jbglwhpt5wvl4h0000gn/T/ipykernel_39770/3968015007.py:6: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_dtree_knear, p_dtree_knear = ttest_rel(acc_dtree, acc_knear)
/var/folders/xb/dnjxhntx75jbglwhpt5wvl4h0000gn/T/ipykernel_39770/3968015007.py:7: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  t_svm_knear, p_svm_knear     = ttest_rel(acc_svm, acc_knear)


### Stacking

After i went trough some Scikit learn docs, i found out about "Stacking", so basically you can add a bunch of models together in a "stack", then fit and predict with them:

In [ ]:
estimators = [
    ('dt', tree.DecisionTreeClassifier()),
    ('svm', svm.SVC()),
    ('kn', KNeighborsClassifier())
]

clf  = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)

clf.fit(x_train, y_train)

,estimators,"[('dt', ...), ('svm', ...), ...]"
,final_estimator,LogisticRegression()
,cv,None
,stack_method,'auto'
,n_jobs,None
,passthrough,False
,verbose,0
,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2


In [78]:
y_pred = clf.predict(x_test)
print("F1:", f1_score(y_pred, y_test, average='macro'))
print("Accuracy:", accuracy_score(y_pred, y_test))


F1: 0.9679484229611699
Accuracy: 0.9856424982053122


### Random Forest

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. Trees in the forest use the best split strategy, i.e. equivalent to passing splitter="best" to the underlying DecisionTreeClassifier. The sub-sample size is controlled with the max_samples parameter if bootstrap=True (default), otherwise the whole dataset is used to build each tree

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [75]:
model_rf = RandomForestClassifier(
    n_estimators=200,      # number of trees
    max_depth=20,          # max depth of each tree
    min_samples_split=5,   # min samples to split a node
    min_samples_leaf=2,    # min samples at a leaf
    max_features='sqrt',   # number of features to consider at each split
    random_state=42,
    n_jobs=-1              # use all CPU cores
)

model_rf.fit(x_train, y_train)

,n_estimators,200
,criterion,'gini'
,max_depth,20
,min_samples_split,5
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [77]:
y_pred = model_rf.predict(x_test)
print("F1:", f1_score(y_pred, y_test, average='macro'))
print("Accuracy:", accuracy_score(y_pred, y_test))


F1: 0.8802204387930586
Accuracy: 0.9526202440775305
